In [1]:
################################################
# Example usages Notebook
################################################
import tensorflow as tf
from utils.loader import DataLoader
from utils import tools, config

## Data Loader Usage

We batch at the file loading level for quicker access to the generator when training. (See loader.py Generator class)

In [2]:
batch_size = 32
loader = DataLoader(batch_size=batch_size) # Instantiate loader
image_ds = loader.get_image_ds().repeat() # Call any of the dataset loaders to get their respective generators
image_masks_ds = loader.get_mask_ds().repeat() # Make them repeat so can reuse
binary_labels_ds = loader.get_binary_ds().repeat()
bounding_boxes_ds = loader.get_bboxes_ds().repeat()

Joining datasets together for training

In [3]:
# Zip together the batched images and their corresponding labels
train_ds = tf.data.Dataset.zip((image_ds, binary_labels_ds))

# Make into a Numpy iterator
train_ds = train_ds.as_numpy_iterator()

Successfully joined dataset generators loaded from file

In [4]:
images, labels = next(train_ds) # Each next call loads a new batched set
print(images.shape, labels.shape)

(32, 256, 256, 3) (32, 1)


To load validation set generators use the 'val' flag

In [5]:
image_val_ds = loader.get_image_ds(val=True)

# Make the dataset generator repeat infinitely
image_val_ds = image_val_ds.repeat().as_numpy_iterator()

To load entire test set of NumpyArrays (Not generators) use 'test_mode' flag

In [ ]:
image_test_ds = loader.get_image_ds(test_mode=True)

# How to train with the loader

Method 1 - Create a generator and pass it to the model.fit() fcn e.g model.fit(generator_img(),...) * BEST METHOD *

In [ ]:
def generator_img():
    ''' Merges together datasets into a unified generator to pass for training '''
    a = image_ds.as_numpy_iterator() # Add the datasets you want to use as numpy iterators
    b = image_masks_ds.as_numpy_iterator()
    c = binary_labels_ds.as_numpy_iterator()
    
    while True:
        X = a.next() # Call next on them every time to yield batches
        Y1 = b.next()
        Y2 = c.next()
        
        # Regularisation and shuffling
        X, Y1, Y2 = tools.get_randomised_data([X, Y1, Y2]) # Use our randomiser and augmentation from tools.py
        X, Y1 = tools.data_augmentation(X, Y1)
        
        yield X, (Y1, Y2) # Return in this format for the MTL -> inputs, outputs